# CRYPTOINSIGHT HUB

In [2]:
import requests
import pandas as pd
import numpy as np
import time

In [ ]:
!pip install arch
!pip install tabulate
!pip install matplotlib seaborn


In [7]:
API_KEY = "CG-e5nkKqYuns3bSAoYNrDqFY8P"   

def fetch_coin_data(coin_id, days=90, max_retries=5):
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
    params = {'vs_currency':'usd','days':days,'interval':'daily'}
    headers = {"x-cg-api-key": API_KEY}

    retries = 0
    while retries < max_retries:
        try:
            response = requests.get(url, params=params, headers=headers, timeout=10)
            response.raise_for_status()
            data = response.json()
            break  # success
        except requests.exceptions.HTTPError as e:
            if response.status_code == 429:
                retries += 1
                wait_time = 2 ** retries  # exponential backoff
                print(f"Rate limit hit, retrying in {wait_time}s...")
                time.sleep(wait_time)
            else:
                raise RuntimeError(f"Failed to fetch data for {coin_id}: {e}")
        except Exception as e:
            raise RuntimeError(f"Failed to fetch data for {coin_id}: {e}")

  # --- Prices (approx OHLC) ---
    prices = pd.DataFrame(data['prices'], columns=['Date', 'Close'])
    prices['Date'] = pd.to_datetime(prices['Date'], unit='ms')
    prices['Open'] = prices['Close']
    prices['High'] = prices['Close']
    prices['Low'] = prices['Close']

    # --- Market Cap ---
    market_cap = pd.DataFrame(data['market_caps'], columns=['Date', 'Market Cap'])
    market_cap['Date'] = pd.to_datetime(market_cap['Date'], unit='ms')

    # --- Volume ---
    volume = pd.DataFrame(data['total_volumes'], columns=['Date', 'Volume'])
    volume['Date'] = pd.to_datetime(volume['Date'], unit='ms')

    # --- Merge ---
    df = prices.merge(market_cap, on='Date').merge(volume, on='Date')
    df['Coin'] = coin_id.capitalize()

    df = df[['Coin', 'Date', 'Open', 'High', 'Low', 'Close', 'Market Cap', 'Volume']]
    df = df.sort_values('Date').reset_index(drop=True)

    return df


## Fetching Coin Data from the API

In [ ]:
btc_df = fetch_coin_data('bitcoin')
time.sleep(2)
eth_df = fetch_coin_data('ethereum')
time.sleep(2)
car_df = fetch_coin_data('cardano')
time.sleep(2)
dog_df = fetch_coin_data('dogecoin')

all_df = pd.concat([btc_df, eth_df, car_df, dog_df], ignore_index=True)

pd.set_option('display.float_format', '{:,.2f}'.format)

print(all_df.head(10).to_markdown(tablefmt="grid"))

#added delay between request inorder to avoide 429 error


### Calculate Realized Volatility

In [5]:
#add_realized_volatility calculates volatility based on the historical data of the asset's price
def add_realized_volatility(df, window=30): #Rolling Standard Deviation → Realized Volatility
    df = df.sort_values("Date")
    df['LogReturn'] = np.log(df['Close'] / df['Close'].shift(1))
    df['RealizedVol'] = df['LogReturn'].rolling(window).std() * np.sqrt(365)  # represents how the market has been over a period of time
    return df

### Calculate ATR

In [10]:
def add_atr(df, window=14): #Average True Range (ATR) → Calc Intraday Volatility
    df = df.sort_values("Date")
    df['H-L'] = df['High'] - df['Low']
    df['H-PC'] = (df['High'] - df['Close'].shift(1)).abs()
    df['L-PC'] = (df['Low'] - df['Close'].shift(1)).abs()
    df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis=1)
    # True Range = max of (High-Low, High-prevClose, Low-prevClose)
    df['ATR'] = df['TR'].rolling(window).mean()# Rolling ATR helps to measure risk and stoploss
    return df

### GARCH implementation

In [4]:
from arch import arch_model

def estimate_garch(df):
    from arch import arch_model
    returns = df['LogReturn'].dropna() * 100  # percentage returns
    am = arch_model(returns, vol='Garch', p=1, q=1)
    res = am.fit(disp='off')
    # create full length series with NaN for first missing returns
    garch_vol = pd.Series(index=df.index, data=np.nan)
    garch_vol.iloc[len(df) - len(res.conditional_volatility):] = (
        res.conditional_volatility / 100  # scale back to decimal
    ) 
    df['garch_vol'] = garch_vol
    return df, res

In [11]:
btc_df = add_realized_volatility(btc_df)
btc_df = add_atr(btc_df)
btc_df, btc_garch = estimate_garch(btc_df)

eth_df = add_realized_volatility(eth_df)
eth_df = add_atr(eth_df)
eth_df, eth_garch = estimate_garch(eth_df)

car_df = add_realized_volatility(car_df)
car_df = add_atr(car_df)
car_df, car_garch = estimate_garch(car_df)

dog_df = add_realized_volatility(dog_df)
dog_df = add_atr(dog_df)
dog_df, dog_garch = estimate_garch(dog_df)



In [13]:
def full_pipeline(coin_id):
    df = fetch_coin_data(coin_id)
    df = add_realized_volatility(df)
    df = add_atr(df)
    df, garch_model = estimate_garch(df)
    return df, garch_model


In [15]:
all_df = pd.concat([btc_df, eth_df, car_df, dog_df], ignore_index=True)
all_df = all_df.sort_values(['Coin', 'Date']).reset_index(drop=True)


In [19]:
cols_to_drop = ["Market Cap", "LogReturn"]

for df in [btc_df, eth_df, car_df, dog_df]:
    for c in cols_to_drop:
        if c in df.columns:
            df.drop(columns=c, inplace=True)


In [20]:
btc_df.head(10)

,Coin,Date,Open,High,Low,Close,Volume,RealizedVol,H-L,H-PC,L-PC,TR,ATR,garch_vol
0,Bitcoin,2025-09-06,"110,662.18","110,662.18","110,662.18","110,662.18","56,020,470,627.41",NaN,0.00,NaN,NaN,0.00,NaN,NaN
1,Bitcoin,2025-09-07,"110,209.19","110,209.19","110,209.19","110,209.19","19,340,758,702.08",NaN,0.00,452.99,452.99,452.99,NaN,0.02
2,Bitcoin,2025-09-08,"111,131.99","111,131.99","111,131.99","111,131.99","22,147,049,159.34",NaN,0.00,922.80,922.80,922.80,NaN,0.02
3,Bitcoin,2025-09-09,"112,025.13","112,025.13","112,025.13","112,025.13","37,159,847,899.86",NaN,0.00,893.14,893.14,893.14,NaN,0.02
4,Bitcoin,2025-09-10,"111,547.44","111,547.44","111,547.44","111,547.44","45,338,021,458.38",NaN,0.00,477.69,477.69,477.69,NaN,0.02
5,Bitcoin,2025-09-11,"113,975.32","113,975.32","113,975.32","113,975.32","52,205,187,155.06",NaN,0.00,"2,427.89","2,427.89","2,427.89",NaN,0.02
6,Bitcoin,2025-09-12,"115,503.17","115,503.17","115,503.17","115,503.17","43,752,917,632.95",NaN,0.00,"1,527.84","1,527.84","1,527.84",NaN,0.02
7,Bitcoin,2025-09-13,"116,160.14","116,160.14","116,160.14","116,160.14","51,192,951,485.86",NaN,0.00,656.98,656.98,656.98,NaN,0.02
8,Bitcoin,2025-09-14,"115,970.58","115,970.58","115,970.58","115,970.58","30,063,858,747.76",NaN,0.00,189.56,189.56,189.56,NaN,0.02
9,Bitcoin,2025-09-15,"115,373.56","115,373.56","115,373.56","115,373.56","26,889,399,414.61",NaN,0.00,597.03,597.03,597.03,NaN,0.02


In [ ]:
print(car_df.head().to_markdown(tablefmt="grid"))

In [16]:
folder = "Data"

btc_df = pd.read_csv(f"{folder}/btc_df_saved.csv")
eth_df = pd.read_csv(f"{folder}/eth_df_saved.csv")
car_df = pd.read_csv(f"{folder}/car_df_saved.csv")
dog_df = pd.read_csv(f"{folder}/dog_df_saved.csv")


In [23]:
for df in [btc_df, eth_df, car_df, dog_df]:
    df['Date'] = pd.to_datetime(df['Date'])


In [ ]:
btc_df.dtypes


In [17]:
import pickle

with open("Data/btc_garch_model.pkl", "rb") as f:
    btc_garch = pickle.load(f)

with open("Data/eth_garch_model.pkl", "rb") as f:
    eth_garch = pickle.load(f)    

with open("Data/car_garch_model.pkl", "rb") as f:
    car_garch = pickle.load(f)    

with open("Data/dog_garch_model.pkl", "rb") as f:
    dog_garch = pickle.load(f)
#arch models cannot be saved with pickle directly unless you extract the model results.
#Use the built-in save() method